In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime


In [6]:
# ------------------------------------
# 1) Define your portfolio tickers
# ------------------------------------

# Define your portfolio tickers
portfolio_ticker = [
    "AAPL", "JPM", "NFLX", "CSCO", "GE", "DHR", "ANET", "UNP", "FI", "VRTX",
    "NVDA", "LLY", "BAC", "MS", "AMD", "INTU", "C", "UBER", "MU", "MRVL",
    "MSFT", "V", "JNJ", "NOW", "ABT", "PLTR", "PFE", "CMCSA", "LMT", "SBUX",
    "GOOG", "XOM", "CRM", "AXP", "GS", "VZ", "LOW", "KKR", "PANW", "MMC",
    "AMZN", "MA", "ABBV", "BX", "DIS", "BKNG", "AMGN", "SCHW", "APP", "NKE",
    "META", "UNH", "CVX", "TMO", "PM", "RTX", "SYK", "TJX", "GILD", "PLD",
    "TSLA", "ORCL", "KO", "ISRG", "ADBE", "T", "NEE", "COP", "BMY", "LRCX",
    "AVGO", "COST", "WFC", "IBM", "CAT", "AMAT", "BSX", "BA", "UPS", "KLAC",
    "BRK-B", "HD", "TMUS", "PEP", "QCOM", "SPGI", "HON", "DE", "GEV", "CHTR",
    "WMT", "PG", "MRK", "MCD", "TXN", "BLK", "PGR", "ADP", "ADI", "CEG"
]


In [9]:
# ------------------------------------
# 3) Fetch data from yfinance
# ------------------------------------
def fetch_data(ticker, interval, period):
    """
    Fetch historical stock data for a given ticker using Yahoo Finance.
    """
    try:
        stock = yf.Ticker(ticker)
        print(f"📥 Fetching historical data for {ticker}...")
        data = stock.history(interval=interval, period=period)  

        if data.empty:
            print(f"⚠️ No data found for {ticker}.")
            return None
        return data

    except Exception as e:
        print(f"❌ Could not fetch data for {ticker}: {e}")
        return None


In [1]:
def fractal_pivots(df, n=2):
    df["PivotHigh"] = np.nan
    df["PivotLow"] = np.nan
    for i in range(n, len(df) - n):
        high_window = df["High"].iloc[i - n: i + n + 1]
        low_window = df["Low"].iloc[i - n: i + n + 1]
        if df["High"].iloc[i] == high_window.max():
            df.at[df.index[i], "PivotHigh"] = df["High"].iloc[i]
        if df["Low"].iloc[i] == low_window.min():
            df.at[df.index[i], "PivotLow"] = df["Low"].iloc[i]
    return df


In [ ]:
def zigzag_indicator(df, zigzag_threshold=0.05):
    df["ZigZag"] = np.nan
    last_pivot = None  
    direction = None  

    for i in range(len(df)):
        if not pd.isna(df["PivotHigh"].iloc[i]) or not pd.isna(df["PivotLow"].iloc[i]):
            current_price = float(df["Close"].iloc[i])
            if last_pivot is None:
                last_pivot = current_price
                df.at[df.index[i], "ZigZag"] = current_price
            else:
                change = (float(current_price) - float(last_pivot)) / float(last_pivot)

                if direction is None:
                    if abs(change) >= zigzag_threshold:
                        direction = "up" if change > 0 else "down"
                        last_pivot = current_price
                        df.at[df.index[i], "ZigZag"] = current_price
                else:
                    if direction == "up" and current_price < last_pivot * (1 - zigzag_threshold):
                        direction = "down"
                        last_pivot = current_price
                        df.at[df.index[i], "ZigZag"] = current_price
                    elif direction == "down" and current_price > last_pivot * (1 + zigzag_threshold):
                        direction = "up"
                        last_pivot = current_price
                        df.at[df.index[i], "ZigZag"] = current_price

    return df


In [3]:
def fib_levels(high, low):
    if pd.isna(high) or pd.isna(low):
        return pd.Series([np.nan] * 5, index=["Fib_0.236", "Fib_0.382", "Fib_0.500", "Fib_0.618", "Fib_0.786"])
    diff = high - low
    return pd.Series({
        "Fib_0.236": high - 0.236 * diff,
        "Fib_0.382": high - 0.382 * diff,
        "Fib_0.500": high - 0.500 * diff,
        "Fib_0.618": high - 0.618 * diff,
        "Fib_0.786": high - 0.786 * diff,
    })


In [4]:
def evaluate_algorithm(interval='60m', period="1mo"):
    failed_tickers = []
    price_data = {}

    for ticker in portfolio_ticker:
        dataF = fetch_data(ticker, interval, period)

        # ✅ Check for missing data
        if dataF is None or dataF.empty or len(dataF) < 2:
            print(f"⚠️ Not enough data for {ticker}. Skipping.")
            failed_tickers.append(ticker)
            continue

        # ✅ Compute Pivot Points & ZigZag
        dataF = fractal_pivots(dataF)
        dataF = zigzag_indicator(dataF)

        # ✅ Compute Swing High/Low
        dataF["SwingHigh"] = dataF["ZigZag"].ffill()
        dataF["SwingLow"] = dataF["ZigZag"].bfill()

        # ✅ Compute Fibonacci Levels
        fib_df = dataF.apply(lambda row: fib_levels(row["SwingHigh"], row["SwingLow"]), axis=1)
        dataF = pd.concat([dataF, fib_df], axis=1)

        # ✅ Store price data
        price_data[ticker] = dataF[["Close", "Fib_0.382", "Fib_0.618", "Fib_0.786"]]

    return price_data


In [12]:
# ------------------------------------
# 5) Prompt for portfolio size
# ------------------------------------
def get_portfolio_size():
    """
    Prompt user for a portfolio size and convert it to float.
    Retries if user does not provide a valid number.
    """
    while True:
        try:
            val = float(input("Enter the value of your portfolio: "))
            return val
        except ValueError:
            print("That's not a number! Try again.\n")

In [ ]:
# ------------------------------------
# 6) Dynamic portfolio alocation following
# the long or short signals for every stock 
# ------------------------------------

def calculate_atr(data, window=14):
    """
    Computes the Average True Range (ATR) for volatility-based risk management.
    """
    high_low = data["High"] - data["Low"]
    high_close = abs(data["High"] - data["Close"].shift())
    low_close = abs(data["Low"] - data["Close"].shift())

    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = true_range.rolling(window=window).mean()
    
    return atr



In [ ]:
def track_portfolio_over_time(price_data, initial_capital):
    """
    Simulates portfolio performance with:
    - ATR-based stop-loss.
    - Fibonacci retracement strategy.
    - Dynamic position sizing.
    """
    portfolio_value_data = []
    portfolio_composition_data = []

    cash_balance = float(initial_capital)
    holdings = {ticker: 0 for ticker in price_data.keys()}
    buy_prices = {ticker: 0 for ticker in price_data.keys()}  
    atr_values = {ticker: calculate_atr(price_data[ticker]) for ticker in price_data.keys()}

    all_dates = sorted(set(date for data in price_data.values() for date in data.index))
    risk_per_trade = 0.02  # Risk 2% of capital per trade

    for date in all_dates:
        date = pd.Timestamp(date)
        total_value = cash_balance
        daily_holdings = {ticker: 0 for ticker in price_data.keys()}

        for ticker, data in price_data.items():
            if date not in data.index:
                continue  # Skip missing data

            close_price = float(data.loc[date, "Close"])
            fib_618 = data.loc[date, "Fib_0.618"]
            fib_786 = data.loc[date, "Fib_0.786"]
            fib_382 = data.loc[date, "Fib_0.382"]
            atr = atr_values[ticker].loc[date] if date in atr_values[ticker].index else None

            if atr is None:
                continue  # Skip if ATR data is unavailable

            stop_loss_level = close_price - (1.5 * atr)
            position_size = (cash_balance * risk_per_trade) / (1.5 * atr)  # ATR-based sizing

            # ✅ Fibonacci-Based Trading Logic
            if holdings[ticker] == 0 and close_price <= fib_618:
                shares_to_buy = position_size / close_price
                holdings[ticker] += shares_to_buy
                buy_prices[ticker] = close_price
                cash_balance -= shares_to_buy * close_price
                print(f"✅ Bought {ticker} at {close_price}")

            elif holdings[ticker] > 0:  
                if close_price > fib_382 or close_price < stop_loss_level:  # ATR-based stop-loss
                    cash_balance += holdings[ticker] * close_price
                    holdings[ticker] = 0
                    print(f"❌ Sold {ticker} at {close_price}")

            total_value += holdings[ticker] * close_price
            daily_holdings[ticker] = holdings[ticker]

        # ✅ Compute Allocation
        allocation = {
            ticker: (holdings[ticker] * close_price) / total_value 
            if total_value > 0 else 0
            for ticker in holdings
        }

        # ✅ Store Portfolio Data
        portfolio_value_data.append({"Date": date, "Portfolio Value": total_value, "Cash Balance": cash_balance})
        portfolio_composition_data.append({
            "Date": date, **daily_holdings, 
            **{f"Alloc_{ticker}": round(alloc, 4) for ticker, alloc in allocation.items()}
        })

    # ✅ Convert to DataFrames
    portfolio_value_df = pd.DataFrame(portfolio_value_data).set_index("Date")
    portfolio_composition_df = pd.DataFrame(portfolio_composition_data).set_index("Date")

    return portfolio_value_df, portfolio_composition_df


In [ ]:
import requests
# Extract free risk rate from the 10-year US Treasury yield
def get_risk_free_rate(API_KEY="0J2S8H8ZNHTX1MRR"):
    url = f"https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={API_KEY}"
    response = requests.get(url).json()

    if "data" not in response:
        print("❌ Error fetching Treasury yield data.")
        return None
    
    # Extract latest 10-year yield (most recent entry)
    latest_yield = float(response["data"][0]["value"]) / 100  # Convert percentage to decimal
    print(f"📉 US 10-Year Treasury Yield (Risk-Free Rate): {latest_yield:.4%}")
    
    return latest_yield 

free_risk_rate = get_risk_free_rate()
free_risk_rate_daily = free_risk_rate / 252  # Daily risk-free rate

In [ ]:
def compute_performance_metrics(portfolio_df):
    """
    Computes Sharpe Ratio, cumulative returns, and drawdown.
    """
    portfolio_df["Daily Return"] = portfolio_df["Portfolio Value"].pct_change() # Not daily returns, but portfolio value changes every t
    portfolio_df["Cumulative Return"] = (1 + portfolio_df["Daily Return"]).cumprod() # Cumulative returns
    
    # Sharpe Ratio Calculation
    excess_daily_return = portfolio_df["Daily Return"] - (free_risk_rate_daily) # Ojo con esto
    sharpe_ratio = excess_daily_return.mean() / excess_daily_return.std() * np.sqrt(252) if excess_daily_return.std() != 0 else np.nan # Asumming a daily frecuency of the portfolio

    # Maximum Drawdown
    cumulative_max = portfolio_df["Cumulative Return"].cummax()
    drawdown = portfolio_df["Cumulative Return"] / cumulative_max - 1
    max_drawdown = drawdown.min()

    print(f"\n📊 Performance Metrics:")
    print(f"🔹 Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"🔹 Max Drawdown: {max_drawdown:.2%}")

    return sharpe_ratio, max_drawdown


In [ ]:
import matplotlib.pyplot as plt

def plot_portfolio_allocation(portfolio_composition_df):
    """
    Plots the evolution of portfolio allocation over time.
    """
    allocation_cols = [col for col in portfolio_composition_df.columns if col.startswith("Alloc_")]
    portfolio_composition_df[allocation_cols].plot(figsize=(12, 6), title="Portfolio Allocation Over Time", stacked=True)
    plt.ylabel("Percentage Allocation")
    plt.xlabel("Date")
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.show()


In [ ]:
def main():
    # Fetch signals and price data
    price_data = evaluate_algorithm()

    # Ask user for initial capital
    initial_capital = get_portfolio_size()

    # Run portfolio tracking with ATR risk management
    portfolio_value_df, portfolio_composition_df = track_portfolio_over_time(price_data, initial_capital)

    # Compute Performance Metrics
    sharpe_ratio, max_drawdown = compute_performance_metrics(portfolio_value_df)

    # Plot Portfolio Allocation Over Time
    plot_portfolio_allocation(portfolio_composition_df)

    # Plot Portfolio Value Over Time
    portfolio_value_df["Portfolio Value"].plot(figsize=(12, 6), title="Portfolio Value Over Time")
    plt.ylabel("Portfolio Value ($)")
    plt.xlabel("Date")
    plt.show()

    # Plot Cumulative Returns
    portfolio_value_df["Cumulative Return"].plot(figsize=(12, 6), title="Cumulative Returns Over Time")
    plt.ylabel("Cumulative Returns")
    plt.xlabel("Date")
    plt.show()

    # Plot every t return
    portfolio_value_df["Daily Return"].plot(figsize=(12, 6), title="Daily Returns Over Time")   
    plt.ylabel("Every t Return")
    plt.xlabel("Date")
    plt.show()

    # ✅ Return DataFrames
    return portfolio_value_df, portfolio_composition_df

# Run the script
portfolio_value_df, portfolio_composition_df = main()


In [16]:
main()

📥 Fetching historical data for AAPL...


$AAPL: possibly delisted; no price data found  (period=1mo)


⚠️ No data found for AAPL.
⚠️ Not enough data for AAPL. Skipping.
📥 Fetching historical data for JPM...


$JPM: possibly delisted; no price data found  (period=1mo)


⚠️ No data found for JPM.
⚠️ Not enough data for JPM. Skipping.
📥 Fetching historical data for NFLX...


...

$ADI: possibly delisted; no price data found  (period=1mo)


⚠️ No data found for ADI.
⚠️ Not enough data for ADI. Skipping.
📥 Fetching historical data for CEG...


$CEG: possibly delisted; no price data found  (period=1mo)


⚠️ No data found for CEG.
⚠️ Not enough data for CEG. Skipping.


KeyError: "None of ['Date'] are in the columns"